In [29]:
## imports
import csv

In [30]:
def calculatePiAiBi(jobs):
    for job in jobs:
        job['pi_ai'] = int(job['processing_time']) / int(job['earliness_penalty']) 
        job['pi_bi'] = int(job['processing_time']) / int(job['tardiness_penalty']) 
    return jobs

In [31]:
def calculateDueDate(jobs, percentage):
    sum = 0
    for job in jobs:
        sum += int(job['processing_time']);
    
    return sum * percentage;

In [32]:
def readJobsFromFile():    
    jobs = []

    spamReader = csv.reader(open('jobs.csv', newline=''), delimiter=',', quotechar='|')
    index = 1
    for row in spamReader:
        job = {}
        job['initial_position'] = index
        job['processing_time'] = row[0]
        job['earliness_penalty'] = row[1]
        job['tardiness_penalty'] = row[2]
        jobs.append(job)
        index += 1

    return jobs

In [33]:
def calculateJobsBeforeDueDate(sortedByPiAiDecreasing,sortedByPiBiIncreasing, dueDateLimit):
    beforeDueDate = []

    sumOfProcessingTime = 0
    for job in sortedByPiAiDecreasing:
        sumOfProcessingTime += int(job['processing_time'])
        if sumOfProcessingTime < dueDateLimit:
            beforeDueDate.append(job)
            sortedByPiBiIncreasing.remove(job)

    beforeDueDate = sorted(beforeDueDate, key=lambda k: k['pi_ai'],reverse=True )

    return beforeDueDate;

In [34]:
def merge(beforeDueDate, sortedByPiBiIncreasing):
    result = beforeDueDate + sortedByPiBiIncreasing
    return result

In [35]:
def getSumOfPenalties(resultsAfterFO):
    sumOfPenalties = 0 

    for job in resultsAfterFO:
        sumOfPenalties += int(job['penalty_time'])
        
    return sumOfPenalties;

In [36]:
def getProcessingTime(resultsAfterFO):
    processingTime = 0 

    for job in resultsAfterFO:
        processingTime += int(job['processing_time'])
        
    return processingTime;

In [37]:
def objectiveFunction(result, dueDateLimit):
    completion_Time = int(dueDateLimit - getProcessingTime(getBeforeDueDate(result, int(dueDateLimit)))) 

    resultAfterFO = [] 
    for job in result:
        completion_Time= completion_Time + int(job['processing_time'])
        if(completion_Time < dueDateLimit):
            job['penalty_time'] = (int(dueDateLimit) - completion_Time) * int(job['earliness_penalty'])
        elif(completion_Time == dueDateLimit):
            job['penalty_time'] = (int(dueDateLimit) -completion_Time)
        elif(completion_Time > dueDateLimit):
            job['penalty_time'] = (completion_Time - int(dueDateLimit)) * int(job['tardiness_penalty'])
        resultAfterFO.append(job)
    
    return resultAfterFO

In [74]:
def constructiveHeuristic(jobs, percentage):
    # passo 02 - calcular o dueDate
    dueDateLimit = calculateDueDate(jobs,percentage);
    #print("due date limit: ", dueDateLimit)
    # passo 03 - calcular o PiAi e o PiBi
    jobs = calculatePiAiBi(jobs)
    # passo 04 - Ordenar
    sortedByPiAiDecreasing = sorted(jobs, key=lambda k: k['pi_ai'], reverse=True )
    sortedByPiBiIncreasing = sorted(jobs, key=lambda k: k['pi_bi'] )
    # passo 05 - calcular o beforeDueDate
    beforeDueDate  = calculateJobsBeforeDueDate(sortedByPiAiDecreasing,sortedByPiBiIncreasing, dueDateLimit)
    # passo 06 - Merge
    result = merge(beforeDueDate, sortedByPiBiIncreasing)
    # passo 07 - Calcular a função objetivo
    resultsAfterFO = objectiveFunction(result, dueDateLimit)
    # passo 08 = mostrar resultados
    sumOfPenalties = getSumOfPenalties(resultsAfterFO)
    sumOfProcessingTime = getProcessingTime(resultsAfterFO)
    
    #print(sumOfPenalties)
    print("sum of Penalties: ", sumOfPenalties)
    print("sum of processing time: ", sumOfProcessingTime)
    
    return resultsAfterFO
    

In [73]:
jobs = readJobsFromFile()
k = 1
for i in range(0,100,10):
    print("----------------- k =", k, "--------------------------" )
    constructiveHeuristic(jobs[i:i+10], 0.2)
    k +=1
    print("--------------------------------------------------\n")
    
    

----------------- k = 1 --------------------------
sum of Penalties:  2152
--------------------------------------------------

----------------- k = 2 --------------------------
sum of Penalties:  1787
--------------------------------------------------

----------------- k = 3 --------------------------
sum of Penalties:  1731
--------------------------------------------------

----------------- k = 4 --------------------------
sum of Penalties:  2411
--------------------------------------------------

----------------- k = 5 --------------------------
sum of Penalties:  1525
--------------------------------------------------

----------------- k = 6 --------------------------
sum of Penalties:  1640
--------------------------------------------------

----------------- k = 7 --------------------------
sum of Penalties:  2674
--------------------------------------------------

----------------- k = 8 --------------------------
sum of Penalties:  1828
------------------------------------

In [75]:
# passo 02 - Quebrar o vetor em 2, Antes do due date e depois do due date

def getBeforeDueDate(listToSplit, dueDate):
    beforeDueDate = []
    completion_time = 0
    
    for job in listToSplit: 
        completion_time = completion_time + int(job['processing_time'])
        if( completion_time <= dueDate):
            beforeDueDate.append(job)
    return beforeDueDate
            
def getAfterDueDate(listToSplit, dueDate):
    afterDueDate = []
    completion_time = 0
    
    for job in listToSplit: 
        completion_time = completion_time + int(job['processing_time'])
        if( completion_time > dueDate):
            afterDueDate.append(job)
    return afterDueDate


print(getBeforeDueDate(listaAfterFO,23))
print(getAfterDueDate(listaAfterFO,23))

[{'initial_position': 4, 'processing_time': '13', 'earliness_penalty': '2', 'tardiness_penalty': '13', 'pi_ai': 6.5, 'pi_bi': 1.0, 'penalty_time': 12}, {'initial_position': 2, 'processing_time': '6', 'earliness_penalty': '1', 'tardiness_penalty': '15', 'pi_ai': 6.0, 'pi_bi': 0.4, 'penalty_time': 0}]
[{'initial_position': 7, 'processing_time': '12', 'earliness_penalty': '5', 'tardiness_penalty': '15', 'pi_ai': 2.4, 'pi_bi': 0.8, 'penalty_time': 180}, {'initial_position': 3, 'processing_time': '13', 'earliness_penalty': '5', 'tardiness_penalty': '13', 'pi_ai': 2.6, 'pi_bi': 1.0, 'penalty_time': 325}, {'initial_position': 6, 'processing_time': '12', 'earliness_penalty': '9', 'tardiness_penalty': '8', 'pi_ai': 1.3333333333333333, 'pi_bi': 1.5, 'penalty_time': 296}, {'initial_position': 9, 'processing_time': '12', 'earliness_penalty': '6', 'tardiness_penalty': '8', 'pi_ai': 2.0, 'pi_bi': 1.5, 'penalty_time': 392}, {'initial_position': 5, 'processing_time': '12', 'earliness_penalty': '7', 't

In [103]:
def getEvaluatedVector(beforeDueDate, afterDueDate, listToCompare, dueDate):
    sumOfProcessingTime = getProcessingTime(beforeDueDate)
    toReturn = listToCompare.copy()
    
    for job in afterDueDate:    
        if int(job['processing_time'])<= dueDate - sumOfProcessingTime:
            beforeDueDate.append(job)
            afterDueDate.remove(job)
            beforeDueDate = sorted(beforeDueDate, key=lambda k: k['pi_ai'],reverse=True)
            
            merged = beforeDueDate + afterDueDate
            applyFO = objectiveFunction(merged,dueDate)
            if getSumOfPenalties(applyFO) < getSumOfPenalties(listToCompare):
                toReturn = applyFO.copy()
            else:
                toReturn = listToCompare.copy()
    return toReturn


In [104]:
#passo 03 - Pegar o ultimo elemento do afterduedate e colocar no before due date


def heuristicaMelhoria(jobs, percentage):
    
    # passo 01 - Pegar o vetor resultante da heuristica 
    
    listaAfterFO = constructiveHeuristic(jobs, percentage)
    
    
    # passo 02- Buscar o DueDate
    
    dueDateLimit = calculateDueDate(listaAfterFO,percentage)    

    
    #passo 02- dividir o vetor em before e after duedate:
    
    beforeDueDate = getBeforeDueDate(listaAfterFO,dueDateLimit)
    
    afterDueDate = getAfterDueDate(listaAfterFO,dueDateLimit)
    
    # passo 03- utilizar os jobs do vetor AfterDuedate na folga do vetor beforeDueDate
    
    resultado = getEvaluatedVector(beforeDueDate, afterDueDate, listaAfterFO, dueDateLimit)
    
    
    print("sum of Penalties: ", getSumOfPenalties(resultado))
    print("sum of processing time: ", getProcessingTime(resultado))
    


In [108]:
toOptimizate = jobs[10:20]

dueDate = int(calculateDueDate(toOptimizate,0.2))

vetor = heuristicaMelhoria(toOptimizate,0.2) 



sum of Penalties:  1787
sum of processing time:  129
sum of Penalties:  1787
sum of processing time:  129
